<a href="https://colab.research.google.com/github/tantanchen/Stream/blob/main/Phi_1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers --quiet
!pip install accelerate --quiet
!pip install einops --quiet

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

In [ ]:
model = AutoModelForCausalLM.from_pretrained("Open-Orca/oo-phi-1_5",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16
    ).to('cuda')
tokenizer = AutoTokenizer.from_pretrained("Open-Orca/oo-phi-1_5",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16)

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-1_5:
- configuration_mixformer_sequential.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-1_5:
- modeling_mixformer_sequential.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
sys_prompt = "I am professional data cleaner. Given any input, I will reformat it only in JSON. I will create a JSON block for every person"
input = "John and Mary Smith 48 Main St, Somerville MA, 09231 Matthew Henner and Julie Smith INC, 827 South Street Springfield, New Jersey 08793 Jonathon L Franklin Melissa H Frank Boissevain Rd New York, 10984"

prefix = "<|im_start|>"
suffix = "<|im_end|>\n"
sys_format = prefix + "system\n" + sys_prompt + suffix
user_format = prefix + "user\n" + input + suffix
assistant_format = prefix + "assistant\n"
input_text = sys_format + user_format + assistant_format

In [ ]:
generation_config = GenerationConfig(
    max_length=1024, temperature=0.01, top_p=0.95, repetition_penalty=1.1,
    do_sample=True, use_cache=True,
    eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.pad_token_id,
    transformers_version="4.33.1"
    )

inputs = tokenizer(input_text, return_tensors="pt", return_attention_mask=False).to('cuda')
outputs = model.generate(**inputs, generation_config=generation_config)

text = tokenizer.batch_decode(outputs)[0]
print(text)

<|im_start|>system
I am professional data cleaner. Given any input, I will reformat it only in JSON. I will create a JSON block for every person<|im_end|>
<|im_start|>user
John and Mary Smith 48 Main St, Somerville MA, 09231 Matthew Henner and Julie Smith INC, 827 South Street Springfield, New Jersey 08793 Jonathon L Franklin Melissa H Frank Boissevain Rd New York, 10984<|im_end|>
<|im_start|>assistant
[
  {
    "name": "John and Mary Smith",
    "address": {
      "street": "48 Main St",
      "city": "Somerville",
      "state": "MA",
      "zip": "09231"
    }
  },
  {
    "name": "Matthew Henner and Julie Smith",
    "address": {
      "street": "INC",
      "city": "827 South Street",
      "state": "Springfield",
      "zip": "08793"
    }
  },
  {
    "name": "Jonathon L Franklin",
    "address": {
      "street": "Boissevain Rd",
      "city": "New York",
      "state": "NY",
      "zip": "10984"
    }
  }
]
<|im_end|>
